# AoC 2023 DAY 2

## Python

In [ ]:
from collections import defaultdict
file = open('input_test.txt', 'r')

def draw_to_dict(draw):
    draw_dict = defaultdict(int)
    for num, color in draw:
        draw_dict[color.strip()]=int(num)
    return draw_dict

games = {}
for line in file:
    game, sets = line.split(": ")
    game = int(game.split(" ")[1])
    sets = [s.split(', ') for s in sets.split("; ")]
    sets = [draw_to_dict([tuple(d.split(" ")) for d in s]) for s in sets]
    games[game]=sets
    
bag = {"red":12, "green":13, "blue":14}

In [ ]:
valid_games_sum = 0
for game in games:
    valid = True
    for g_set in games[game]:
        for color in bag:
            if bag[color] < g_set[color]:
                valid = False
                break
    if valid:
        valid_games_sum += game

{"part_1": valid_games_sum}

In [ ]:
power_sum = 0
for game in games:
    mincubes =defaultdict(int)
    for g_set in games[game]:
        for color in g_set:
            mincubes[color] = max(g_set[color], mincubes[color])
    power_sum += mincubes["red"]*mincubes["green"]*mincubes["blue"]

{"part_2": power_sum}

## Cypher and python
using `graphdatascience` python client for neo4J

In [ ]:
from graphdatascience import GraphDataScience

In [ ]:
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

In [ ]:
for game in games:
    for draw, g_set in enumerate(games[game]):
        for color in g_set:
            query ="""
MERGE (g:Game {{id:{game}}})
MERGE (d:Draw {{game:{game}, draw:{draw}}})
MERGE (c:Color {{color:'{color}'}})
MERGE (g)-[:HAS_DRAW]->(d)
MERGE (d)-[:HAS_CUBES_OF_COLOR {{num:{num}}}]->(c);
            """.format(
                game=game,
                draw=draw,
                color=color,
                num=g_set[color])
            gds.run_cypher(query)

In [ ]:
for color in bag:
    gds.run_cypher(
        """
    MERGE (b:Bag)
    WITH b
    MATCH (c:Color {{color:'{color}'}})
    MERGE (b)-[:HAS_CUBES {{num:{num}}}]->(c);
        """.format(color=color, num=bag[color])
    )

In [ ]:
gds.run_cypher(
        """
MATCH (g:Game)
WHERE NOT EXISTS {
    (g)-->(d:Draw)-[hcc]->(:Color)<-[hc WHERE hc.num<hcc.num]-(:Bag)
}
RETURN sum(g.id) AS part1
        """
    )

In [ ]:
gds.run_cypher(
        """
MATCH (g:Game)-->(d:Draw)-[r:HAS_CUBES_OF_COLOR]->(c)
WITH g, c, max(r.num) AS max_by_color
WITH g, collect(max_by_color) AS maxs_by_color
WITH g, reduce(acc = 1, mbc IN  maxs_by_color| acc * mbc) AS prod_maxs
RETURN sum(prod_maxs) AS part2
        """
    )